In [1]:
import torch

In [3]:
from deep_learning.models.feature_extractors.resnet import Resnet50

model= Resnet50()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

random_input = torch.randn(200, 3, 256, 256).to(device)

model.eval()
with torch.no_grad():
    features = model(random_input)

print(features.shape)

cuda
torch.Size([200, 2048, 1, 1])


In [ ]:
from deep_learning.models.feature_extractors.efficientnet import EfficientNetB1

model= EfficientNetB1()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

random_input = torch.randn(200, 3, 256, 256).to(device)

model.eval()
with torch.no_grad():
    features = model(random_input)

print(features.shape)

c:\Users\user\.julia\conda\3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.julia\conda\3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda
torch.Size([200, 1280, 1, 1])
